In [ ]:
!pip install gradio langchain langchain-community transformers sentencepiece pdfplumber --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import pdfplumber
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Hugging Face LLM
llm_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", max_length=512)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Prompt template
prompt = PromptTemplate(
    input_variables=["resume"],
    template="""
You are a professional career coach. Analyze the following resume and give:
1. Summary of skills
2. Strengths
3. Areas for improvement
4. Suggestions to improve

Resume:
{resume}
"""
)

chain = LLMChain(llm=llm, prompt=prompt)

# Resume processing function
def analyze_resume(file):
    with pdfplumber.open(file.name) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    result = chain.invoke({"resume": text})
    return result['text']


Device set to use cpu


In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=analyze_resume,
    inputs=gr.File(label="Upload your Resume (PDF)", file_types=[".pdf"]),
    outputs=gr.Textbox(label="AI Resume Analysis", lines=20),
    title="🧠 AI Resume Analyzer",
    description="Upload your resume (PDF), and let an AI career coach analyze it!"
)

interface.launch(share=True)  # use share=True to get a public URL


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ca5dd6223d44fb09c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
